In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from functools import reduce
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
all_2020president = pd.read_csv('sos/allcounties-2020presidentfinal.csv')
all_2020senate = pd.read_csv('sos/allcounties-2020senatefinal.csv')
all_2018senate = pd.read_csv('sos/allcounties-2018senate.csv')
all_2018gov = pd.read_csv('sos/allcounties-2018gov.csv')
all_2016president = pd.read_csv('sos/allcounties-2016president.csv')
for df in [all_2020president,all_2020senate,all_2018senate,all_2018gov,all_2016president]:
    for col in df.columns:
        if col != 'County' and col != 'Partial':
            df[col] = [float(str(x).replace(',','').strip('%')) for x in df[col]]
tx = reduce(lambda x, y: pd.merge(x, y, on = 'County'), [all_2020president,all_2020senate,all_2018senate,all_2018gov,all_2016president])
droplist =[]
for col in tx:
    if 'Unnamed' in col:
        droplist.append(col)
tx = tx.drop(columns=droplist)

/Users/dahliadry/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def tabulate_district(district,overlaps):
    try:
        counties = pd.read_csv(district+'/counties.csv')
    except:
        counties = pd.read_excel(district+'/counties.xlsx')
    try:
        counties['County'] = [x.strip().upper() for x in counties['County']]
        votes = counties[counties['Partial'] == 'No'].merge(tx,how='left',on='County')
    except:
        counties['Counties'] = [x.strip().upper() for x in counties['Counties']]
        votes = counties[counties['Partial'] == 'No'].merge(tx,how='left',left_on='Counties',right_on='County')
    droplist =[]
    for col in votes:
        if 'Unnamed' in col:
            droplist.append(col)
    votes = votes.drop(columns=droplist)
    #votes.to_csv(district+'/district_votetotals.csv')
    overlaps.append(votes)
    def intersect(x,y):
        intersect = np.intersect1d(x.columns,y.columns).tolist()
        try:
            intersect.remove('Precinct')
        except:
            pass
        return intersect
    tx23 = reduce(lambda x, y: pd.merge(x, y,on = intersect(x,y),how='outer',suffixes=('', '_y')),overlaps)
    tx23 = tx23.drop(tx23.filter(regex='_y$').columns.tolist(),axis=1)
    cty = tx23.pop('County')
    partial = tx23.pop('Partial')
    tx23.insert(0,'County',cty)
    tx23.insert(1,'Partial',partial)
    #display(tx23)
    return tx23
def aggbydistrict(pcts, datalist,county,intpcts=True,current=True,agg=True):
    if intpcts:
        pcts['Precinct'] = [int(x) for x in pcts['Precinct']]
    for i in range(len(datalist)):
        for col in datalist[i].columns:
            if 'Unnamed' in col:
                datalist[i] = datalist[i].drop(columns=col)
    data = reduce(lambda x, y: pd.merge(x, y,on = 'Precinct',how='outer'),datalist)
    district = pcts.merge(data,on='Precinct')
    if current:
        district = district.drop_duplicates(subset=['Precinct','CD','Trump_2020','Biden_2020'])
    else:
        district = district.drop_duplicates(subset=['Precinct','CD'])
    for col in district.columns:
        if intpcts:
            if col != 'CD':
                district[col] = [float(str(x).replace(',','').replace('%','')) for x in district[col]]
            else:
                district[col] = [int(str(x).replace(',','').replace('%','')) for x in district[col]]
        else:
            if col != 'Precinct' and col != 'CD':
                district[col] = [float(str(x).replace(',','').replace('%','')) for x in district[col]]
            elif col == 'CD':
                district[col] = [int(str(x).replace(',','').replace('%','')) for x in district[col]]
    #district.to_csv('test.csv')
    if agg:
        district= district.groupby('CD').agg('sum')
        droplist =['Precinct']
        for col in district:
            if 'Unnamed' in str(col):
                droplist.append(col)
        try:
            district = district.drop(columns=droplist)
        except:
            pass
    else:
        droplist =[]
        for col in district:
            if 'Unnamed' in str(col):
                droplist.append(col)
        try:
            district = district.drop(columns=droplist)
        except:
            pass
    district['County'] = [county for x in district[district.columns[0]]]
    district['Partial'] = ['Yes' for x in district[district.columns[0]]]
    #display(district)
    return district

## Aggregate Partial Counties (Precinct ->District)

In [3]:
#EL Paso 2020,2018,2016
elpasopcts= pd.read_csv('partial-counties/elpaso/elpaso-pcts.csv')
elpasodata = [pd.read_csv('partial-counties/elpaso/elpaso-2020presidentfinal.csv'),
              pd.read_csv('partial-counties/elpaso/elpaso-2020senatefinal.csv'),
              pd.read_csv('partial-counties/elpaso/elpaso-2018senate.csv'),
              pd.read_csv('partial-counties/elpaso/elpaso-2018gov.csv'),
              pd.read_csv('partial-counties/elpaso/elpaso-2016president.csv')]
elpasodistricts = aggbydistrict(elpasopcts,elpasodata, 'EL PASO')
#Bexar 2016,2018,2020
bexarpcts= pd.read_csv('partial-counties/bexar/bexar-pcts.csv')
bexardata = [pd.read_csv('partial-counties/bexar/bexar-2016president.csv'),
             pd.read_csv('partial-counties/bexar/bexar-2018senate.csv'),
            pd.read_csv('partial-counties/bexar/bexar-2018gov.csv')]
bexardistricts = aggbydistrict(bexarpcts,bexardata, 'BEXAR',current=False)
bexar2020pcts= pd.read_csv('partial-counties/bexar/bexar-precincts2020.csv')
bexar2020data = [pd.read_csv('partial-counties/bexar/bexar-2020president.csv'),
             pd.read_csv('partial-counties/bexar/bexar-2020senate.csv')]
bexar2020districts = aggbydistrict(bexar2020pcts,bexar2020data, 'BEXAR')
#La Salle 2020
lasallepcts=pd.read_csv('partial-counties/lasalle/lasalle-pcts.csv')
lasalledata = [pd.read_csv('partial-counties/lasalle/lasalle-2020.csv')]
lasalledistricts = aggbydistrict(lasallepcts,lasalledata, 'LA SALLE')

#Dallas 2020,2018,2016 
dallas2020pcts=pd.read_excel('partial-counties/dallas/dallas-pcts2020.xlsx')
dallas2020data = [pd.read_csv('partial-counties/dallas/dallas-2020presidentfinal.csv'),
                  pd.read_excel('partial-counties/dallas/dallas_2020senatefinal.xlsx')]
dallas2020districts = aggbydistrict(dallas2020pcts,dallas2020data, 'DALLAS',intpcts = False)

dallas2018pcts=pd.read_excel('partial-counties/dallas/dallas-pcts2018.xlsx')
dallas2018data = [pd.read_csv('partial-counties/dallas/dallas-2018senate.csv'),
                  pd.read_csv('partial-counties/dallas/dallas-2018gov.csv')]
dallas2018districts = aggbydistrict(dallas2018pcts,dallas2018data, 'DALLAS',intpcts = False,current=False)

dallas2016pcts=pd.read_excel('partial-counties/dallas/dallas-pcts2016.xlsx')
dallas2016data = [pd.read_csv('partial-counties/dallas/dallas-2016president.csv')]
dallas2016districts = aggbydistrict(dallas2016pcts,dallas2016data, 'DALLAS',intpcts = False,current=False)

#Denton 2018,2016,2020
dentonpcts = pd.read_excel('partial-counties/denton/denton-pcts.xlsx')
dentondata = [pd.read_csv('partial-counties/denton/denton-2018senate.csv'),
              pd.read_csv('partial-counties/denton/denton-2018governor.csv'),
             pd.read_csv('partial-counties/denton/denton-2016president.csv'),
             pd.read_csv('partial-counties/denton/denton-president2020final.csv'),
             pd.read_csv('partial-counties/denton/denton-2020senatefinal.csv')]
dentondistricts = aggbydistrict(dentonpcts,dentondata, 'DENTON')

#Tarrant 2016,2018,2020
tarrantpcts= pd.read_excel('partial-counties/tarrant/tarrant-pcts.xlsx')
tarrantdata = [pd.read_csv('partial-counties/tarrant/tarrant-senate2018.csv'),
               pd.read_csv('partial-counties/tarrant/tarrant-gov2018.csv'),
              pd.read_csv('partial-counties/tarrant/tarrant-2020senatefinal.csv'),
               pd.read_csv('partial-counties/tarrant/tarrant-2020presidentfinal.csv'),
              pd.read_csv('partial-counties/tarrant/tarrant-president2016.csv')]
tarrantdistricts = aggbydistrict(tarrantpcts,tarrantdata, 'TARRANT')

#Fort Bend 2016,2018,2020
fortbendpcts= pd.read_excel('partial-counties/fortbend/fortbend-pcts.xlsx')
fortbenddata = [pd.read_csv('partial-counties/fortbend/fortbend-2020presidentfinal.csv'),
                pd.read_csv('partial-counties/fortbend/fortbend-2020senatefinal.csv'),
               pd.read_csv('partial-counties/fortbend/fortbend-2018senate.csv'),
                pd.read_csv('partial-counties/fortbend/fortbend-2018governor.csv'),
               pd.read_csv('partial-counties/fortbend/fortbend-2016president.csv')]
fortbenddistricts = aggbydistrict(fortbendpcts,fortbenddata, 'FORT BEND')

#Brazoria 2016,2018,2020
brazoriapcts= pd.read_csv('partial-counties/brazoria/brazoria-pcts.csv')
brazoriadata = [pd.read_csv('partial-counties/brazoria/brazoria-2020president.csv'),
                pd.read_csv('partial-counties/brazoria/brazoria-2020senate.csv'),
               pd.read_csv('partial-counties/brazoria/brazoria-2018senate.csv'),
                pd.read_csv('partial-counties/brazoria/brazoria-2018gov.csv'),
               pd.read_csv('partial-counties/brazoria/brazoria-2016president.csv')]
brazoriadistricts = aggbydistrict(brazoriapcts,brazoriadata, 'BRAZORIA')

#Harris 2016,2018,2020
harrispcts = pd.read_excel('partial-counties/harris/harris-pcts.xlsx')
harrisdata = [pd.read_csv('partial-counties/harris/harris-2020presidentfinal.csv'),
              pd.read_csv('partial-counties/harris/harris-2020senatefinal.csv'),
             pd.read_csv('partial-counties/harris/harris-senate2018.csv'),
              pd.read_csv('partial-counties/harris/harris-gov2018.csv'),
             pd.read_csv('partial-counties/harris/harris-2016president.csv')]
harrisdistricts = aggbydistrict(harrispcts,harrisdata,'HARRIS')

#Bastrop 2016,2018,2020
bastroppcts = pd.read_excel('partial-counties/bastrop/bastrop-pcts.xlsx')
bastropdata = [pd.read_csv('partial-counties/bastrop/bastrop-2016president.csv'),
               pd.read_csv('partial-counties/bastrop/bastrop-2018senate.csv'),
              pd.read_csv('partial-counties/bastrop/bastrop-2018gov.csv'),
               pd.read_csv('partial-counties/bastrop/bastrop-2020senatefinal.csv'),
              pd.read_csv('partial-counties/bastrop/bastrop-2020presidentfinal.csv')]
bastropdistricts = aggbydistrict(bastroppcts,bastropdata,'BASTROP')

#Lee 2018,2016,2020
leepcts = pd.read_csv('partial-counties/lee/lee-pcts.csv')
leedata = [pd.read_csv('partial-counties/lee/lee-president2016.csv'),
           pd.read_csv('partial-counties/lee/lee-2018.csv'),
          pd.read_csv('partial-counties/lee/lee-2020.csv')]
leedistricts = aggbydistrict(leepcts,leedata,'LEE',current=False)

#Comal 2016,2018,2020
comalpcts = pd.read_csv('partial-counties/comal/comal-pcts.csv')
comaldata = [pd.read_csv('partial-counties/comal/comal-2016president.csv'),
             pd.read_csv('partial-counties/comal/comal-2018governor.csv'),
            pd.read_csv('partial-counties/comal/comal-senate2018.csv'),
            pd.read_csv('partial-counties/comal/comal-2020all.csv')]
comaldistricts = aggbydistrict(comalpcts,comaldata,'COMAL')

#Hays 2016,2018,2020
hayspcts = pd.read_excel('partial-counties/hays/hays-pcts.xlsx')
haysdata = [pd.read_csv('partial-counties/hays/hays-2016president.csv'),
            pd.read_csv('partial-counties/hays/hays-2018gov.csv'),
           pd.read_csv('partial-counties/hays/hays-2018senate.csv'),
            pd.read_csv('partial-counties/hays/hays-president2020final.csv'),
           pd.read_csv('partial-counties/hays/hays-senate2020final.csv')]
haysdistricts= aggbydistrict(hayspcts,haysdata,'HAYS')

#Erath 2016,2018,2020
erathpcts = pd.read_excel('partial-counties/erath/erath-pcts.xlsx')
erathdata = [pd.read_csv('partial-counties/erath/erath-2018all.csv'),
             pd.read_csv('partial-counties/erath/erath-2016president.csv'),
            pd.read_csv('partial-counties/erath/erath-2020all.csv')]
erathdistricts = aggbydistrict(erathpcts,erathdata, 'ERATH')

#Bell 2016,2018,2020
bellpcts = pd.read_csv('partial-counties/bell/bell-pcts.csv')
belldata = [pd.read_csv('partial-counties/bell/bell-2020president.csv'),
            pd.read_csv('partial-counties/bell/bell-2020senate.csv'),
           pd.read_csv('partial-counties/bell/bell-2018senate.csv'),
            pd.read_csv('partial-counties/bell/bell-2016president.csv')]
belldistricts = aggbydistrict(bellpcts,belldata,'BELL')

#Travis 2020
travispcts = pd.read_csv('partial-counties/travis/travis-pcts.csv')
travisdata = [pd.read_csv('partial-counties/travis/travis-2020allfinal.csv').drop(columns='CD')]
travisdistricts = aggbydistrict(travispcts,travisdata,'TRAVIS',intpcts=False)

#Collin 2020
collinpcts = pd.read_csv('partial-counties/collin/collin-pcts.csv')
collindata = [pd.read_csv('partial-counties/collin/collin-president2020final.csv'),
             pd.read_csv('partial-counties/collin/collin-senate2020final.csv')]
collindistricts = aggbydistrict(collinpcts,collindata,'COLLIN')

## TX23

In [16]:
#bexar2020districts.to_csv('test.csv')
#El Paso, La Salle, Bexar (3/3 partial counties added)
overlaps = [pd.DataFrame(bexardistricts.loc[23]).swapaxes(0,1),pd.DataFrame(elpasodistricts.loc[23]).swapaxes(0,1),
           pd.DataFrame(lasalledistricts.loc[23]).swapaxes(0,1),pd.DataFrame(bexar2020districts.loc[23]).swapaxes(0,1)]
tx23 = tabulate_district('tx23',overlaps)
tx23 = tx23.groupby(['County']).agg('sum')
tx23.loc["Total"] = tx23.sum(numeric_only=True)
display(tx23)
tx23.to_csv('tx23/tx23_votetotals.csv')

,Trump_2016,Clinton_2016,Cruz_2018,O'Rourke_2018,Abbott_2018,Valdez_2018,Trump_2020,Biden_2020,Cornyn_2020,Hegar_2020,Registered Voters_2018Senate,Total Ballots_2018Senate,Registered Voters_2018Governor,Total Ballots_2018Governor,Total Ballots_2016President,Total Ballots_2020President,Total Ballots_2020Senate,Registered Voters_2020President,Turnout_2020President,Registered Voters_2020Senate,Turnout_2020Senate,Turnout_2018Senate,Turnout_2018Governor,Registered Voters_2016President,Turnout_2016President
County,,,,,,,,,,,,,,,,,,,,,,,,,
BEXAR,54407.0,52678.0,52332.0,56249.0,59447.0,47765.0,76319.0,83553.0,79113.0,75987.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
BREWSTER,2077.0,1873.0,1879.0,2147.0,2033.0,1933.0,2461.0,2258.0,2453.0,2167.0,7292.0,4086.0,7292.0,4062.0,4252.0,4822.0,4772.0,7524.0,64.09,7524.0,63.42,56.03,55.70,7369.0,57.70
CRANE,1049.0,299.0,836.0,213.0,842.0,197.0,1247.0,241.0,1213.0,229.0,2666.0,1055.0,2666.0,1047.0,1384.0,1503.0,1479.0,2663.0,56.44,2663.0,55.54,39.57,39.27,2679.0,51.66
CROCKETT,980.0,372.0,928.0,340.0,921.0,333.0,1220.0,344.0,1146.0,314.0,2497.0,1276.0,2497.0,1266.0,1395.0,1574.0,1493.0,2473.0,63.65,2473.0,60.37,51.10,50.70,2553.0,54.64
CULBERSON,280.0,454.0,297.0,521.0,324.0,422.0,415.0,438.0,367.0,385.0,1722.0,827.0,1722.0,757.0,767.0,864.0,778.0,1709.0,50.56,1709.0,45.52,48.03,43.96,1762.0,43.53
DIMMIT,974.0,2173.0,840.0,2042.0,1042.0,1787.0,1384.0,2264.0,1274.0,2122.0,7378.0,2894.0,7378.0,2851.0,3225.0,3666.0,3468.0,7341.0,49.94,7341.0,47.24,39.22,38.64,7440.0,43.35
EDWARDS,746.0,303.0,604.0,145.0,635.0,100.0,893.0,168.0,824.0,144.0,1449.0,757.0,1449.0,742.0,1073.0,1066.0,986.0,1499.0,71.11,1499.0,65.78,52.24,51.21,1514.0,70.87
EL PASO,3178.0,17059.0,3000.0,15185.0,3801.0,13833.0,6858.0,17317.0,6098.0,16447.0,50376.0,18277.0,50376.0,17879.0,21319.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
FRIO,1856.0,2444.0,1636.0,2016.0,1844.0,1739.0,2823.0,2422.0,2489.0,2283.0,8606.0,3680.0,8606.0,3634.0,4400.0,5279.0,4921.0,8984.0,58.76,8984.0,54.78,42.76,42.23,9130.0,48.19


## TX21

In [17]:
#Bexar,Comal,Hays,Travis (4/4 partial counties added)
overlaps = [
            pd.DataFrame(bexar2020districts.loc[21]).swapaxes(0,1),
            pd.DataFrame(comaldistricts.loc[21]).swapaxes(0,1),
           pd.DataFrame(haysdistricts.loc[21]).swapaxes(0,1),
           pd.DataFrame(travisdistricts.loc[21]).swapaxes(0,1)]
tx21 = tabulate_district('tx21',overlaps)
#tx21 = tx21.groupby(['County']).agg('sum')
tx21.loc["Total"] = tx21.sum(numeric_only=True)
display(tx21)
tx21.to_csv('tx21/tx21_votetotals.csv')

,County,Partial,Trump_2020,Biden_2020,Hegar_2020,Cornyn_2020,Registered Voters_2016President,Trump_2016,Clinton_2016,Total Ballots_2016President,Registered Voters_2018Governor,Abbott_2018,Valdez_2018,Total Ballots_2018_x,Registered Voters_2018Senate,Cruz_2018,O'Rourke_2018,Turnout_2016President,Total Ballots_2018Governor,Turnout_2018Governor,Total Ballots_2018Senate,Turnout_2018Senate,Total Ballots_2020President,Registered Voters_2020President,Turnout_2020President,Total Ballots_2020Senate,Registered Voters_2020Senate,Turnout_2020Senate
0,BEXAR,Yes,68364.0,72610.0,65461.0,72755.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,COMAL,Yes,51423.0,17794.0,16047.0,52154.0,70801.0,37946.0,10283.0,50354.0,58159.0,39507.0,10162.0,50470,58159.0,37451.0,12304.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HAYS,Yes,12942.0,14887.0,13955.0,13453.0,31014.0,9668.0,8733.0,20404.0,34818.0,10885.0,10357.0,NaN,34818.0,9591.0,12075.0,1089.30,22121.0,991.14,22121.0,991.14,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAVIS,Yes,28255.0,94171.0,88494.0,31694.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,REAL,No,1643.0,320.0,286.0,1626.0,2522.0,1382.0,262.0,1681.0,2590.0,1341.0,213.0,NaN,2590.0,1311.0,245.0,66.65,1566.0,60.46,1564.0,60.39,1982.0,2702.0,73.35,1939.0,2702.0,71.76
5,KERR,No,20879.0,6524.0,5946.0,21022.0,34644.0,17727.0,4681.0,23297.0,35854.0,17566.0,4268.0,NaN,35854.0,16822.0,5198.0,67.25,22132.0,61.73,22205.0,61.93,27745.0,37726.0,73.54,27543.0,37726.0,73.01
6,BANDERA,No,10057.0,2505.0,2291.0,9957.0,15524.0,8163.0,1726.0,10218.0,15869.0,7866.0,1602.0,NaN,15869.0,7643.0,1865.0,65.82,9588.0,60.42,9584.0,60.39,12715.0,17098.0,74.37,12537.0,17098.0,73.32
7,GILLESPIE,No,12514.0,3176.0,2996.0,12543.0,18968.0,10446.0,2288.0,13214.0,19294.0,10351.0,2030.0,NaN,19294.0,9890.0,2572.0,69.66,12545.0,65.02,12543.0,65.01,15850.0,20404.0,77.68,15779.0,20404.0,77.33
8,KENDALL,No,20083.0,6020.0,5335.0,20584.0,29077.0,15700.0,3643.0,20120.0,30774.0,16076.0,3382.0,NaN,30774.0,15292.0,4340.0,69.20,19744.0,64.16,19796.0,64.33,26452.0,33836.0,78.18,26392.0,33836.0,78.00
9,BLANCO,No,5443.0,1911.0,1816.0,5457.0,8033.0,4212.0,1244.0,5685.0,8504.0,4329.0,1351.0,NaN,8504.0,4181.0,1570.0,70.77,5795.0,68.14,5808.0,68.30,7441.0,9344.0,79.63,7408.0,9344.0,79.28


## TX 24

In [18]:
# Dallas, Denton, Tarrant (all partial counties added)
overlaps = [pd.DataFrame(tarrantdistricts.loc[24]).swapaxes(0,1),pd.DataFrame(dallas2020districts.loc[24]).swapaxes(0,1),
           pd.DataFrame(dallas2018districts.loc[24]).swapaxes(0,1),pd.DataFrame(dallas2016districts.loc[24]).swapaxes(0,1),
            pd.DataFrame(dentondistricts.loc[24]).swapaxes(0,1)]
tx24 = tabulate_district('tx24',overlaps)
tx24 = tx24.groupby(['County']).agg('sum')
tx24.loc["Total"] = tx24.sum()
display(tx24)
tx24.to_csv('tx24/tx24_votetotals.csv')

,Cruz_2018,O'Rourke_2018,Total Ballots_2018Senate,Abbott_2018,Valdez_2018,Cornyn_2020,Hegar_2020,Trump_2020,Biden_2020,Trump_2016,Clinton_2016,Total Ballots_2016President,Registered Voters_2018Senate,Registered Voters_2018Governor,Total Ballots_2018Governor,Total Ballots_2020President,Registered Voters_2020President,Turnout_2020President,Total Ballots_2020Senate,Registered Voters_2020Senate,Turnout_2020Senate,Turnout_2018Senate,Turnout_2018Governor,Turnout_2016President
County,,,,,,,,,,,,,,,,,,,,,,,,
DALLAS,44639.0,66216.0,111777.0,51910.0,56863.0,61099.0,79051.0,56413.0,85623.0,49152.0,60966.0,115598.0,194037.0,194037.0,111100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DENTON,19884.0,24293.0,44547.0,22571.0,20736.0,28809.0,31061.0,27070.0,34341.0,21706.0,21503.0,0.0,79080.0,79080.0,44344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TARRANT,63011.0,46275.0,110232.0,69248.0,38369.0,81785.0,55030.0,78064.0,60553.0,69270.0,40403.0,116127.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Total,127534.0,136784.0,266556.0,143729.0,115968.0,171693.0,165142.0,161547.0,180517.0,140128.0,122872.0,231725.0,273117.0,273117.0,155444.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## TX22

In [19]:
#Harris, Brazoria, fort Bend (3/3 partial counties added)
overlaps = [pd.DataFrame(harrisdistricts.loc[22]).swapaxes(0,1),
           pd.DataFrame(brazoriadistricts.loc[22]).swapaxes(0,1),
           pd.DataFrame(fortbenddistricts.loc[22]).swapaxes(0,1)]
tx22 = tabulate_district('tx22',overlaps)
#tx22 = tx22.groupby(['County']).agg('sum')
tx22.loc["Total"] = tx22.sum(numeric_only=True)
display(tx22)
tx22.to_csv('tx22/tx22_votetotals.csv')

,County,Partial,Trump_2020,Biden_2020,Cornyn_2020,Hegar_2020,Total Ballots_2018Senate,Registered Voters_2018Senate,Turnout_2018Senate,Cruz_2018,O'Rourke_2018,Total Ballots_2018Governor,Registered Voters_2018Governor,Turnout_2018Governor,Abbott_2018,Valdez_2018,Total Ballots_2016President,Registered Voters_2016President,Turnout_2016President,Trump_2016,Clinton_2016,Total Ballots_2020Senate,Registered Voters_2020Senate,Turnout_2020Senate,Total Ballots_President2016,Registered Voters_President2016,Turnout_President2016,Total Ballots_2020President,Registered Voters_2020President,Turnout_2020President
0,HARRIS,Yes,21912.0,21720.0,21975.0,20283.0,32114.0,60155.0,1350.74,15123.0,16416.0,32105.0,60155.0,1350.42,16737.0,14619.0,35141.0,57074,1579.84,16790.0,16053.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BRAZORIA,Yes,45629.0,42183.0,46506.0,39698.0,63948.0,114030.0,1632.22,32800.0,30485.0,63930.0,114030.0,1631.80,35983.0,26634.0,NaN,NaN,NaN,36219.0,28073.0,89520,127407,2324.36,68455,104145,1917.1,NaN,NaN,NaN
2,FORT BEND,Yes,142144.0,141881.0,145577.0,128612.0,203787.0,338054.0,NaN,101652.0,100749.0,203352.0,338054.0,NaN,113434.0,87014.0,206526.0,NaN,NaN,106636.0,91229.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total,NaN,NaN,209685.0,205784.0,214058.0,188593.0,299849.0,512239.0,2982.96,149575.0,147650.0,299387.0,512239.0,2982.22,166154.0,128267.0,241667.0,NaN,NaN,159645.0,135355.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


## TX10

In [4]:
#Travis, Bastrop, Lee, Harris (4/4 partial counties added)
overlaps = [pd.DataFrame(bastropdistricts.loc[10]).swapaxes(0,1),
           pd.DataFrame(leedistricts.loc[10]).swapaxes(0,1),
           pd.DataFrame(harrisdistricts.loc[10]).swapaxes(0,1),
           pd.DataFrame(travisdistricts.loc[10]).swapaxes(0,1)]
tx10 = tabulate_district('tx10',overlaps)
tx10.loc["Total"] = tx10.sum(numeric_only=True)
display(tx10)
tx10.to_csv('tx10/tx10_votetotals.csv')

,County,Partial,Trump_2016,Clinton_2016,Total Ballots_2016President,Registered Voters_2018Senate,Cruz_2018,O'Rourke_2018,Total Ballots_2018Senate,Registered Voters_2018Governor,Abbott_2018,Valdez_2018,Total Ballots_2018Governor,Cornyn_2020,Hegar_2020,Trump_2020,Biden_2020,Turnout_2018Senate,Turnout_2018Governor,Registered Voters_2016President,Turnout_2016President,Total Ballots_2020President,Registered Voters_2020President,Turnout_2020President,Total Ballots_2020Senate,Registered Voters_2020Senate,Turnout_2020Senate
0,BASTROP,Yes,9176.0,6235.0,16446.0,27593.0,8500.0,7269.0,15964.0,27593.0,9227.0,6294.0,15922.0,11516.0,8911.0,11577.0,9144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LEE,Yes,3350.0,963.0,NaN,NaN,2830.0,888.0,NaN,NaN,3112.0,773.0,NaN,4102.0,1170.0,4125.0,1220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HARRIS,Yes,75445.0,38403.0,121731.0,196007.0,70497.0,43626.0,115706.0,196007.0,75422.0,38013.0,115678.0,102919.0,60687.0,101390.0,65714.0,2950.01,2949.39,175175.0,3489.05,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAVIS,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41083.0,100874.0,37369.0,107413.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WASHINGTON,No,10945.0,3382.0,14828.0,23253.0,10134.0,3263.0,13492.0,23253.0,10532.0,2730.0,13470.0,12988.0,4017.0,12959.0,4261.0,58.02,57.93,22951.0,64.61,17427.0,23947.0,72.77,17280.0,23947.0,72.16
5,FAYETTE,No,8743.0,2144.0,11174.0,16626.0,8228.0,2198.0,10479.0,16626.0,8576.0,1777.0,10461.0,10056.0,2634.0,10171.0,2661.0,63.03,62.92,16279.0,68.64,12941.0,17398.0,74.38,12864.0,17398.0,73.94
6,COLORADO,No,6325.0,1987.0,8513.0,13938.0,5779.0,1825.0,7637.0,13938.0,5992.0,1558.0,7613.0,7367.0,2362.0,7472.0,2420.0,54.79,54.62,13871.0,61.37,9975.0,14378.0,69.38,9890.0,14378.0,68.79
7,AUSTIN,No,9637.0,2320.0,12274.0,19406.0,8722.0,2241.0,11047.0,19406.0,9009.0,1897.0,11029.0,11454.0,2752.0,11447.0,2951.0,56.93,56.83,19263.0,63.72,14554.0,20293.0,71.72,14481.0,20293.0,71.36
8,WALLER,No,10531.0,5748.0,16784.0,32584.0,10167.0,6335.0,16603.0,32584.0,10639.0,5791.0,16600.0,14219.0,7725.0,14260.0,8191.0,50.95,50.95,31132.0,53.91,22734.0,35116.0,64.74,22458.0,35116.0,63.95
Total,NaN,NaN,134152.0,61182.0,201750.0,329407.0,124857.0,67645.0,190928.0,329407.0,132509.0,58833.0,190773.0,215704.0,191132.0,210770.0,203975.0,3233.73,3232.64,278671.0,3801.30,77631.0,111132.0,352.99,76973.0,111132.0,350.20


## TX6

In [21]:
#Tarrant (1/1 partial counties added)
overlaps = [pd.DataFrame(tarrantdistricts.loc[6]).swapaxes(0,1)]
tx6 = tabulate_district('tx6',overlaps)
tx6.loc["Total"] = tx6.sum(numeric_only=True)
display(tx6)
tx6.to_csv('tx6/tx6_votetotals.csv')

,County,Partial,Cruz_2018,O'Rourke_2018,Total Ballots_2018Senate,Abbott_2018,Valdez_2018,Total_2018Governor,Cornyn_2020,Hegar_2020,Trump_2020,Biden_2020,Trump_2016,Clinton_2016,Total Ballots_2016President,Total Ballots_2020President,Registered Voters_2020President,Turnout_2020President,Total Ballots_2020Senate,Registered Voters_2020Senate,Turnout_2020Senate,Registered Voters_2018Senate,Turnout_2018Senate,Total Ballots_2018Governor,Registered Voters_2018Governor,Turnout_2018Governor,Registered Voters_2016President,Turnout_2016President
0,TARRANT,Yes,80877.0,101120.0,183486.0,90131.0,89456.0,182985,107357.0,124192.0,104417.0,131837.0,92010.0,95017.0,196405.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ELLIS,No,41022.0,19106.0,60589.0,43647.0,15879.0,NaN,57078.0,25780.0,56717.0,27565.0,44941.0,16253.0,63673.0,85493.0,120188.0,71.13,85039.0,120188.0,70.75,108349.0,55.92,60501.0,108349.0,55.84,102235.0,62.28
2,NAVARRO,No,10391.0,3918.0,14416.0,10818.0,3411.0,NaN,13701.0,4821.0,13800.0,5101.0,11994.0,4002.0,16433.0,19123.0,29959.0,63.83,18962.0,29959.0,63.29,28641.0,50.33,14408.0,28641.0,50.31,27786.0,59.14
Total,NaN,NaN,132290.0,124144.0,258491.0,144596.0,108746.0,NaN,178136.0,154793.0,174934.0,164503.0,148945.0,115272.0,276511.0,104616.0,150147.0,134.96,104001.0,150147.0,134.04,136990.0,106.25,74909.0,136990.0,106.15,130021.0,121.42


## TX2

In [22]:
# Harris (1/1 Partial Counties Added)
overlaps = [pd.DataFrame(harrisdistricts.loc[2]).swapaxes(0,1)]
tx2 = tabulate_district('tx2',overlaps)
display(tx2)
tx2.to_csv('tx2/tx2_votetotals.csv')

,County,Partial,Trump_2020,Biden_2020,Cornyn_2020,Hegar_2020,Total Ballots_2018Senate,Registered Voters_2018Senate,Turnout_2018Senate,Cruz_2018,O'Rourke_2018,Total Ballots_2018Governor,Registered Voters_2018Governor,Turnout_2018Governor,Abbott_2018,Valdez_2018,Total Ballots_2016President,Registered Voters_2016President,Turnout_2016President,Trump_2016,Clinton_2016,Total Ballots_2020President,Registered Voters_2020President,Turnout_2020President,Total Ballots_2020Senate,Registered Voters_2020Senate,Turnout_2020Senate
0,HARRIS,Yes,174980,170430,180504,157925,266111,454393,8792.07,132559,129460,265844,454393,8782.93,146593,112508,285742,429260,9960.77,145530,119659,NaN,NaN,NaN,NaN,NaN,NaN


## TX31

TX31 Totals at https://docs.google.com/spreadsheets/d/1SQmuQom98rAnIyd4pmC4TBAZFq-q4NHoQ4z7WIH3rfA/edit#gid=0

## TX25

In [ ]:
#Tarrant,Hays,Travis,Bell,Erath (5/5 partial counties added)
overlaps = [pd.DataFrame(tarrantdistricts.loc[25]).swapaxes(0,1),
           pd.DataFrame(haysdistricts.loc[25]).swapaxes(0,1),
           pd.DataFrame(belldistricts.loc[25]).swapaxes(0,1),
           pd.DataFrame(erathdistricts.loc[25]).swapaxes(0,1),
           pd.DataFrame(travisdistricts.loc[25]).swapaxes(0,1)]
tx25 = tabulate_district('tx25',overlaps)
tx25.loc["Total"] = tx25.sum(numeric_only=True)
display(tx25)
tx25.to_csv('tx25/tx25_votetotals.csv')

## TX3

In [24]:
#Collin (0/1 partial counties added)
overlaps = [pd.DataFrame(collindistricts.loc[3]).swapaxes(0,1)]
tx3 = tabulate_district('tx3',overlaps)
#tx32.loc["Total"] = tx32.sum(numeric_only=True)
display(tx3)
tx3.to_csv('tx3/tx3_votetotals.csv')

,County,Partial,Trump_2020,Biden_2020,Cornyn_2020,Hegar_2020,Total Ballots_2020President,Registered Voters_2020President,Turnout_2020President,Total Ballots_2020Senate,Registered Voters_2020Senate,Turnout_2020Senate,Cruz_2018,O'Rourke_2018,Total Ballots_2018Senate,Registered Voters_2018Senate,Turnout_2018Senate,Abbott_2018,Valdez_2018,Total Ballots_2018Governor,Registered Voters_2018Governor,Turnout_2018Governor,Trump_2016,Clinton_2016,Total Ballots_2016President,Registered Voters_2016President,Turnout_2016President
0,COLLIN,Yes,214359,209859,224921,188092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## TX32

In [25]:
#Collin, Dallas (2/2 partial counties added)
overlaps = [pd.DataFrame(dallas2020districts.loc[32]).swapaxes(0,1),
            pd.DataFrame(dallas2018districts.loc[32]).swapaxes(0,1),
           pd.DataFrame(dallas2016districts.loc[32]).swapaxes(0,1),
           pd.DataFrame(collindistricts.loc[32]).swapaxes(0,1)]
tx32 = tabulate_district('tx32',overlaps)
tx32.loc["Total"] = tx32.sum(numeric_only=True)
display(tx32)
tx32.to_csv('tx32/tx32_votetotals.csv')

,County,Partial,Trump_2020,Biden_2020,Cornyn_2020,Hegar_2020,Registered Voters_2018Senate,Cruz_2018,O'Rourke_2018,Total Ballots_2018Senate,Registered Voters_2018Governor,Abbott_2018,Valdez_2018,Total Ballots_2018Governor,Registered Voters_2016President,Trump_2016,Clinton_2016,Total Ballots_2016President,Total Ballots_2020President,Registered Voters_2020President,Turnout_2020President,Total Ballots_2020Senate,Registered Voters_2020Senate,Turnout_2020Senate,Turnout_2018Senate,Turnout_2018Governor,Turnout_2016President
0,DALLAS,Yes,136549.0,175546.0,149507.0,159495.0,194037,44639,66216,111777,194037,51910,56863,111100,409288,117758,127828,258036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,COLLIN,Yes,15395.0,12373.0,15650.0,11159.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total,NaN,NaN,151944.0,187919.0,165157.0,170654.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## TX7

In [26]:
#Harris (1/1 partial counties added)
overlaps = [pd.DataFrame(harrisdistricts.loc[7]).swapaxes(0,1)]
tx7 = tabulate_district('tx7',overlaps)
#tx32.loc["Total"] = tx32.sum(numeric_only=True)
display(tx7)
tx7.to_csv('tx7/tx7_votetotals.csv')

,County,Partial,Trump_2020,Biden_2020,Cornyn_2020,Hegar_2020,Total Ballots_2018Senate,Registered Voters_2018Senate,Turnout_2018Senate,Cruz_2018,O'Rourke_2018,Total Ballots_2018Governor,Registered Voters_2018Governor,Turnout_2018Governor,Abbott_2018,Valdez_2018,Total Ballots_2016President,Registered Voters_2016President,Turnout_2016President,Trump_2016,Clinton_2016,Total Ballots_2020President,Registered Voters_2020President,Turnout_2020President,Total Ballots_2020Senate,Registered Voters_2020Senate,Turnout_2020Senate
0,HARRIS,Yes,143176,170060,152741,154670,246364,413329,8637.83,112286,130185,246082,413329,8627.28,127666,111568,264639,394183,9746.22,121204,124722,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import plotly.express as px

priors = pd.read_excel('regression-presidential.xlsx')
fig = px.scatter(priors, x="Trump_2016", y="Trump_2020",color_discrete_sequence=['red'],hover_data=["County"],trendline="ols")
fig.update_layout(title='Trump 2016 v Trump 2020')
fig.show()
results = px.get_trendline_results(fig)
print(results.px_fit_results.iloc[0].summary())

fig2 = px.scatter(priors, x="Clinton_2016", y="Biden_2020",hover_data=["County"],trendline="ols")
fig2.update_layout(title="Clinton 2016 v Biden 2020")
fig2.show()
results = px.get_trendline_results(fig2)
print(results.px_fit_results.iloc[0].summary())

FileNotFoundError: [Errno 2] No such file or directory: 'regression-presidential.xlsx'

In [67]:
def trump2020(trump2016):
    return 1.2466*trump2016+77.9780
def biden2020(clinton2016):
    return 1.3248*clinton2016+291.9869
estimate = pd.read_excel('projections.xlsx')
for i in range(len(estimate)):
    estimate['Trump_2020'].iloc[i] = trump2020(estimate['Trump_2016'].iloc[i])
    estimate['Biden_2020'].iloc[i] = biden2020(estimate['Clinton_2016'].iloc[i])
estimate.to_excel('projections.xlsx')

/Users/dahliadry/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Estimate TX10

In [68]:
#Travis, Bastrop, Lee, Harris (2/4 partial counties complete) using projected Travis, Lee 2020 counts
overlaps = [pd.DataFrame(bastropdistricts.loc[10]).swapaxes(0,1),
           pd.DataFrame(harrisdistricts.loc[10]).swapaxes(0,1)]
overlaps.append(estimate[(estimate['County']=='Lee') & (estimate['CD']==10)])
overlaps.append(estimate[(estimate['County']=='Travis') & (estimate['CD']==10)])
tx10 = tabulate_district('tx10',overlaps)
tx10.loc["Total"] = tx10.sum(numeric_only=True)
display(tx10)

,County,Partial,Trump_2016,Clinton_2016,Total Ballots_2016President,Registered Voters_2018Senate,Cruz_2018,O'Rourke_2018,Total Ballots_2018Senate,Registered Voters_2018Governor,Abbott_2018,Valdez_2018,Total Ballots_2018Governor,Registered Voters_2020Senate,Cornyn_2020,Hegar_2020,Total Ballots_2020Senate,Registered Voters_2020President,Trump_2020,Biden_2020,Total Ballots_2020President,Turnout_2018Senate,Turnout_2018Governor,Registered Voters_2016President,Turnout_2016President,Unnamed: 0,CD,Turnout_2020President,Turnout_2020Senate
0,BASTROP,Yes,9176.0,6235.0,16446.0,27593.0,8500.0,7269.0,15964.0,27593.0,9227.0,6294.0,15922.0,30209.0,11499.0,8898.0,20987.0,30209.0,11561.000,9130.0000,21140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HARRIS,Yes,75445.0,38403.0,121731.0,196007.0,70497.0,43626.0,115706.0,196007.0,75422.0,38013.0,115678.0,NaN,102162.0,60217.0,166457.0,NaN,100612.000,65206.0000,168257.0,2950.01,2949.39,175175.0,3489.05,NaN,NaN,NaN,NaN
2,Lee,NaN,3350.0,963.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4254.088,1567.7693,NaN,NaN,NaN,NaN,NaN,0.0,10.0,NaN,NaN
3,Travis,NaN,30760.0,74802.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38423.394,99389.6765,NaN,NaN,NaN,NaN,NaN,5.0,10.0,NaN,NaN
4,WASHINGTON,No,10945.0,3382.0,14828.0,23253.0,10134.0,3263.0,13492.0,23253.0,10532.0,2730.0,13470.0,23947.0,12979.0,4013.0,17265.0,23947.0,12949.000,4254.0000,17410.0,58.02,57.93,22951.0,64.61,NaN,NaN,72.70,72.10
5,FAYETTE,No,8743.0,2144.0,11174.0,16626.0,8228.0,2198.0,10479.0,16626.0,8576.0,1777.0,10461.0,17398.0,10047.0,2623.0,12844.0,17398.0,10163.000,2650.0000,12924.0,63.03,62.92,16279.0,68.64,NaN,NaN,74.28,73.82
6,COLORADO,No,6325.0,1987.0,8513.0,13938.0,5779.0,1825.0,7637.0,13938.0,5992.0,1558.0,7613.0,14378.0,7336.0,2345.0,9838.0,14378.0,7440.000,2403.0000,9912.0,54.79,54.62,13871.0,61.37,NaN,NaN,68.94,68.42
7,AUSTIN,No,9637.0,2320.0,12274.0,19406.0,8722.0,2241.0,11047.0,19406.0,9009.0,1897.0,11029.0,20293.0,11294.0,2732.0,14297.0,20293.0,11282.000,2931.0000,14368.0,56.93,56.83,19263.0,63.72,NaN,NaN,70.80,70.45
8,WALLER,No,10531.0,5748.0,16784.0,32584.0,10167.0,6335.0,16603.0,32584.0,10639.0,5791.0,16600.0,35116.0,14162.0,7672.0,22345.0,35116.0,14206.000,8130.0000,22617.0,50.95,50.95,31132.0,53.91,NaN,NaN,64.41,63.63
Total,NaN,NaN,164912.0,135984.0,201750.0,329407.0,122027.0,66757.0,190928.0,329407.0,129397.0,58060.0,190773.0,141341.0,169479.0,88500.0,264033.0,141341.0,210890.482,195661.4458,266628.0,3233.73,3232.64,278671.0,3801.30,5.0,20.0,351.13,348.42


## Estimate TX21

In [69]:
#Bexar,Comal,Hays,Travis (1/4 partial counties complete) using projected 2020 Travis, Bexar,Comal counts
overlaps = [pd.DataFrame(haysdistricts.loc[21]).swapaxes(0,1)]
overlaps.append(estimate[(estimate['County']=='Bexar') & (estimate['CD']==21)])
overlaps.append(estimate[(estimate['County']=='Travis') & (estimate['CD']==21)])
overlaps.append(estimate[(estimate['County']=='Comal') & (estimate['CD']==21)])
print(estimate[(estimate['County']=='Comal') & (estimate['CD']==21)])
tx21 = tabulate_district('tx21',overlaps)
tx21.loc["Total"] = tx21.sum(numeric_only=True)
display(tx21)

   Unnamed: 0 County  CD  Trump_2016  Clinton_2016  Trump_2020  Biden_2020
2           2  Comal  21       37946         10283  47381.4616  13914.9053


,County,Partial,Total Ballots_2016President,Registered Voters_2016President,Turnout_2016President,Trump_2016,Clinton_2016,Total Ballots_2018Governor,Registered Voters_2018Governor,Turnout_2018Governor,Abbott_2018,Valdez_2018,Total Ballots_2018Senate,Registered Voters_2018Senate,Turnout_2018Senate,Cruz_2018,O'Rourke_2018,Trump_2020,Biden_2020,Total Ballots_2020President,Registered Voters_2020President,Cornyn_2020,Hegar_2020,Total Ballots_2020Senate,Registered Voters_2020Senate,Unnamed: 0,CD,Turnout_2020President,Turnout_2020Senate
0,HAYS,Yes,20404.0,31014.0,1089.30,9668.0,8733.0,22121.0,34818.0,991.14,10885.0,10357.0,22121.0,34818.0,991.14,9591.0,12075.0,12891.0000,14818.0000,28469.0,38612.0,13402.0,13888.0,28469.0,38612.0,NaN,NaN,NaN,NaN
1,Bexar,NaN,NaN,NaN,NaN,59830.0,49932.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74662.0560,66441.9005,NaN,NaN,NaN,NaN,NaN,NaN,1.0,21.0,NaN,NaN
2,Travis,NaN,NaN,NaN,NaN,23261.0,69736.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29075.1406,92678.2397,NaN,NaN,NaN,NaN,NaN,NaN,6.0,21.0,NaN,NaN
3,Comal,NaN,NaN,NaN,NaN,37946.0,10283.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47381.4616,13914.9053,NaN,NaN,NaN,NaN,NaN,NaN,2.0,21.0,NaN,NaN
4,REAL,No,1681.0,2522.0,66.65,1382.0,262.0,1566.0,2590.0,60.46,1341.0,213.0,1564.0,2590.0,60.39,1311.0,245.0,1643.0000,320.0000,1982.0,2702.0,1626.0,286.0,1939.0,2702.0,NaN,NaN,73.35,71.76
5,KERR,No,23297.0,34644.0,67.25,17727.0,4681.0,22132.0,35854.0,61.73,17566.0,4268.0,22205.0,35854.0,61.93,16822.0,5198.0,20858.0000,6510.0000,27710.0,37726.0,21000.0,5933.0,27505.0,37726.0,NaN,NaN,73.45,72.91
6,BANDERA,No,10218.0,15524.0,65.82,8163.0,1726.0,9588.0,15869.0,60.42,7866.0,1602.0,9584.0,15869.0,60.39,7643.0,1865.0,10050.0000,2503.0000,12703.0,17098.0,9950.0,2291.0,12530.0,17098.0,NaN,NaN,74.30,73.28
7,GILLESPIE,No,13214.0,18968.0,69.66,10446.0,2288.0,12545.0,19294.0,65.02,10351.0,2030.0,12543.0,19294.0,65.01,9890.0,2572.0,12495.0000,3163.0000,15817.0,20404.0,12522.0,2985.0,15747.0,20404.0,NaN,NaN,77.52,77.18
8,KENDALL,No,20120.0,29077.0,69.20,15700.0,3643.0,19744.0,30774.0,64.16,16076.0,3382.0,19796.0,30774.0,64.33,15292.0,4340.0,20064.0000,6008.0000,26419.0,33836.0,20561.0,5325.0,26358.0,33836.0,NaN,NaN,78.08,77.90
9,BLANCO,No,5685.0,8033.0,70.77,4212.0,1244.0,5795.0,8504.0,68.14,4329.0,1351.0,5808.0,8504.0,68.30,4181.0,1570.0,5429.0000,1905.0000,7419.0,9344.0,5441.0,1812.0,7391.0,9344.0,NaN,NaN,79.40,79.10


## Estimate TX23 (Now Complete 11/20/20)

In [70]:
#El Paso, La Salle, Bexar (2/3 partial counties complete) using projected 2020 Bexar count
overlaps = [pd.DataFrame(elpasodistricts.loc[23]).swapaxes(0,1),
           pd.DataFrame(lasalledistricts.loc[23]).swapaxes(0,1)]
overlaps.append(estimate[(estimate['County']=='Bexar') & (estimate['CD']==23)])
tx23 = tabulate_district('tx23',overlaps)
tx23.loc["Total"] = tx23.sum(numeric_only=True)
display(tx23)

,County,Partial,Trump_2020,Biden_2020,Total Ballots_2020President,Registered Voters_2020President,Cornyn_2020,Hegar_2020,Total Ballots_2020Senate,Registered Voters_2018Senate,Cruz_2018,O'Rourke_2018,Total Ballots_2018Senate,Registered Voters_2018Governor,Abbott_2018,Valdez_2018,Total Ballots_2018Governor,Trump_2016,Clinton_2016,Total Ballots_2016President,Unnamed: 0,CD,Turnout_2020President,Registered Voters_2020Senate,Turnout_2020Senate,Turnout_2018Senate,Turnout_2018Governor,Registered Voters_2016President,Turnout_2016President
0,EL PASO,Yes,6700.0000,16634.0000,23768.0,53580.0,5948.0,15776.0,23009.0,50376.0,3000.0,15185.0,18277.0,50376.0,3801.0,13833.0,17879.0,3178.0,17059.0,21319.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LA SALLE,Yes,349.0000,224.0000,580.0,NaN,220.0,226.0,464.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bexar,NaN,67901.7442,70079.8013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54407.0,52678.0,NaN,3.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HUDSPETH,No,771.0000,371.0000,1157.0,2085.0,673.0,443.0,1153.0,1922.0,509.0,407.0,933.0,1922.0,538.0,335.0,886.0,503.0,324.0,871.0,NaN,NaN,55.49,2085.0,55.30,48.54,46.10,1840.0,47.34
4,CULBERSON,No,415.0000,438.0000,864.0,1709.0,367.0,385.0,778.0,1722.0,297.0,521.0,827.0,1722.0,324.0,422.0,757.0,280.0,454.0,767.0,NaN,NaN,50.56,1709.0,45.52,48.03,43.96,1762.0,43.53
5,REEVES,No,2249.0000,1394.0000,3683.0,7558.0,1978.0,1337.0,3430.0,7089.0,1128.0,1255.0,2398.0,7089.0,1210.0,1123.0,2357.0,1417.0,1659.0,3184.0,NaN,NaN,48.73,7558.0,45.38,33.83,33.25,7103.0,44.83
6,JEFF DAVIS,No,783.0000,501.0000,1304.0,1670.0,766.0,483.0,1285.0,1719.0,683.0,466.0,1177.0,1719.0,719.0,407.0,1165.0,695.0,1.0,1191.0,NaN,NaN,78.08,1670.0,76.95,68.47,67.77,1655.0,71.96
7,PRESIDIO,No,721.0000,1463.0000,2217.0,4789.0,651.0,1374.0,2091.0,4884.0,436.0,1221.0,1667.0,4884.0,465.0,1157.0,1652.0,652.0,1458.0,2208.0,NaN,NaN,46.29,4789.0,43.66,34.13,33.82,4806.0,45.94
8,BREWSTER,No,2451.0000,2251.0000,4805.0,7524.0,2447.0,2160.0,4761.0,7292.0,1879.0,2147.0,4086.0,7292.0,2033.0,1933.0,4062.0,2077.0,1873.0,4252.0,NaN,NaN,63.86,7524.0,63.28,56.03,55.70,7369.0,57.70
9,LOVING,No,60.0000,4.0000,66.0,111.0,58.0,5.0,66.0,121.0,47.0,6.0,54.0,121.0,47.0,9.0,56.0,58.0,4.0,65.0,NaN,NaN,59.46,111.0,59.46,44.63,46.28,110.0,59.09


## Estimate TX24 (Now Complete 11/9/20)

In [71]:
# Dallas, Denton, Tarrant (all partial counties added) using projected 2020 counts for Denton
overlaps = [pd.DataFrame(tarrantdistricts.loc[24]).swapaxes(0,1),
            pd.DataFrame(dallas2020districts.loc[24]).swapaxes(0,1),
           pd.DataFrame(dallas2018districts.loc[24]).swapaxes(0,1),
            pd.DataFrame(dallas2016districts.loc[24]).swapaxes(0,1)]
overlaps.append(estimate[(estimate['County']=='Denton') & (estimate['CD']==24)])
tx24 = tabulate_district('tx24',overlaps)
tx24 = tx24.groupby(['County']).agg('sum')
tx24.loc["Total"] = tx24.sum()
display(tx24)

,Cruz_2018,O'Rourke_2018,Total Ballots_2018Senate,Abbott_2018,Valdez_2018,Registered Voters_2020Senate,Cornyn_2020,Hegar_2020,Total Ballots_2020Senate,Registered Voters_2020President,Trump_2020,Biden_2020,Total Ballots_2020President,Trump_2016,Clinton_2016,Total Ballots_2016President,Unnamed: 0,CD,Turnout_2020President,Turnout_2020Senate,Turnout_2018Senate,Turnout_2018Governor,Turnout_2016President
County,,,,,,,,,,,,,,,,,,,,,,,
DALLAS,44639.0,66216.0,111777.0,51910.0,56863.0,205051.0,61043.0,78955.0,143686.0,205051.0,56356.0000,85521.0000,144085.0,49152.0,60966.0,115598.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Denton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27136.6776,28779.1613,0.0,21706.0,21503.0,0.0,4.0,24.0,0.0,0.0,0.0,0.0,0.0
TARRANT,63011.0,46275.0,110232.0,69248.0,38369.0,189767.0,81354.0,54622.0,139397.0,189767.0,77644.0000,60101.0000,140107.0,69270.0,40403.0,116127.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Total,107650.0,112491.0,222009.0,121158.0,95232.0,394818.0,142397.0,133577.0,283083.0,394818.0,161136.6776,174401.1613,284192.0,140128.0,122872.0,231725.0,4.0,24.0,0.0,0.0,0.0,0.0,0.0


## Estimate TX25

In [72]:
#Tarrant,Hays,Travis,Bell,Erath (4/5 partial counties added) using projected 2020 counts for Travis, Erath
overlaps = [pd.DataFrame(tarrantdistricts.loc[25]).swapaxes(0,1),
           pd.DataFrame(haysdistricts.loc[25]).swapaxes(0,1),
           pd.DataFrame(belldistricts.loc[25]).swapaxes(0,1)]
overlaps.append(estimate[(estimate['County']=='Erath') & (estimate['CD']==25)])
overlaps.append(estimate[(estimate['County']=='Travis') & (estimate['CD']==25)])
tx25 = tabulate_district('tx25',overlaps)
tx25.loc["Total"] = tx25.sum(numeric_only=True)
display(tx25)

,County,Partial,Cruz_2018,O'Rourke_2018,Total Ballots_2018Senate,Abbott_2018,Valdez_2018,Total_2018Governor,Registered Voters_2020Senate,Cornyn_2020,Hegar_2020,Total Ballots_2020Senate,Registered Voters_2020President,Trump_2020,Biden_2020,Total Ballots_2020President,Trump_2016,Clinton_2016,Total Ballots_2016President,Registered Voters_2016President,Turnout_2016President,Total Ballots_2018Governor,Registered Voters_2018Governor,Turnout_2018Governor,Registered Voters_2018Senate,Turnout_2018Senate,Total Registered_2016President,Unnamed: 0,CD,Turnout_2020President,Turnout_2020Senate
0,TARRANT,Yes,1868.0,913.0,2807.0,1959.0,789.0,2799,5349.0,2413.0,1131.0,3641.0,5349.0,2374.0000,1231.0000,3676.0,2131.0,778.0,3048.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HAYS,Yes,14504.0,11870.0,26905.0,15998.0,9924.0,NaN,44000.0,20290.0,14383.0,35704.0,44000.0,19397.0000,15522.0000,35704.0,14202.0,9160.0,25398.0,34195.0,921.25,26905.0,38200.0,846.48,38200.0,846.48,NaN,NaN,NaN,NaN,NaN
2,BELL,Yes,218.0,131.0,355.0,NaN,NaN,NaN,23991.0,3520.0,7770.0,11762.0,23991.0,3472.0000,8068.0000,11828.0,239.0,108.0,364.0,NaN,NaN,NaN,NaN,NaN,591.0,NaN,559,NaN,NaN,NaN,NaN
3,Erath,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6370.8148,2100.3389,NaN,5048.0,1365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,25.0,NaN,NaN
4,Travis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63047.4838,117352.6397,NaN,50513.0,88361.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,25.0,NaN,NaN
5,BURNET,No,13859.0,4444.0,18490.0,14497.0,3712.0,NaN,33697.0,18642.0,5299.0,24468.0,33697.0,18721.0000,5615.0000,24646.0,14638.0,3797.0,19204.0,29587.0,64.91,18549.0,31072.0,59.70,31072.0,59.51,NaN,NaN,NaN,73.14,72.61
6,LAMPASAS,No,5836.0,1569.0,7470.0,6073.0,1343.0,NaN,15424.0,8086.0,1969.0,10304.0,15424.0,8070.0000,2134.0000,10373.0,6385.0,1483.0,8205.0,13786.0,59.52,7517.0,14099.0,53.32,14099.0,52.98,NaN,NaN,NaN,67.25,66.80
7,CORYELL,No,10626.0,5067.0,15863.0,11281.0,4392.0,NaN,41450.0,15331.0,7096.0,23182.0,41450.0,15397.0000,7542.0000,23425.0,12225.0,5064.0,18253.0,37644.0,48.49,15931.0,38635.0,41.23,38635.0,41.06,NaN,NaN,NaN,56.51,55.93
8,HAMILTON,No,2795.0,507.0,3330.0,2903.0,405.0,NaN,5832.0,3572.0,593.0,4274.0,5832.0,3613.0000,641.0000,4347.0,3060.0,479.0,3620.0,5467.0,66.22,3338.0,5611.0,59.49,5611.0,59.35,NaN,NaN,NaN,74.54,73.29
9,BOSQUE,No,5718.0,1374.0,7155.0,5948.0,1104.0,NaN,12724.0,7341.0,1483.0,9006.0,12724.0,7446.0000,1552.0000,9094.0,6339.0,1278.0,7838.0,12002.0,65.31,7158.0,12209.0,58.63,12209.0,58.60,NaN,NaN,NaN,71.47,70.78


## Estimate TX32 (Now complete 11/9/20)

In [74]:
#Collin, Dallas (1/2 partial counties added) using projected 2020 counts for Collin
overlaps = [pd.DataFrame(dallas2020districts.loc[32]).swapaxes(0,1),
            pd.DataFrame(dallas2018districts.loc[32]).swapaxes(0,1),
           pd.DataFrame(dallas2016districts.loc[32]).swapaxes(0,1)]
overlaps.append(estimate[(estimate['County']=='Collin') & (estimate['CD']==32)])
tx32 = tabulate_district('tx32',overlaps)
tx32.loc["Total"] = tx32.sum(numeric_only=True)
display(tx32)

,County,Partial,Registered Voters_2020President,Trump_2020,Biden_2020,Total Ballots_2020President,Registered Voters_2020Senate,Cornyn_2020,Hegar_2020,Total Ballots_2020Senate,Registered Voters_2018Senate,Cruz_2018,O'Rourke_2018,Total Ballots_2018Senate,Registered Voters_2018Governor,Abbott_2018,Valdez_2018,Total Ballots_2018Governor,Registered Voters_2016President,Trump_2016,Clinton_2016,Total Ballots_2016President,Unnamed: 0,CD,Turnout_2020President,Turnout_2020Senate,Turnout_2018Senate,Turnout_2018Governor,Turnout_2016President
0,DALLAS,Yes,442518,136431.0000,175313.0000,316610,442518,149368,159305,316211,194037,44639,66216,111777,194037,51910,56863,111100,409288,117758.0,127828.0,258036,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Collin,NaN,NaN,14966.1218,9654.3485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11943.0,7067.0,NaN,10.0,32.0,NaN,NaN,NaN,NaN,NaN
Total,NaN,NaN,NaN,151397.1218,184967.3485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129701.0,134895.0,NaN,10.0,32.0,0.0,0.0,0.0,0.0,0.0
